In [1]:
import os
from pathlib import Path
import urllib.request
import zipfile

from tqdm import tqdm

from traccuracy import run_metrics
from traccuracy.loaders.ctc import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics, DivisionMetrics

In [2]:
url = "http://data.celltrackingchallenge.net/training-datasets/Fluo-N2DL-HeLa.zip"
data_dir = 'downloads'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

filename = url.split('/')[-1]
file_path = os.path.join(data_dir, filename)
ds_name = filename.split('.')[0]

In [19]:
# Add a utility to make a progress bar when downloading the file
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

if not os.path.exists(file_path):
    print(f"Downloading {ds_name} data from the CTC website")
    # Downloading data
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, file_path, reporthook=t.update_to)
    # Unzip the data
    # TODO add a progress bar to zip as well
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

Fluo-N2DL-HeLa.zip: 191MB [00:10, 17.4MB/s]                              


In [3]:

gt_data = load_ctc_data('downloads/Fluo-N2DL-HeLa/01_GT/TRA', 'man_track.txt')
pred_data = load_ctc_data('sample-data/Fluo-N2DL-HeLa/01_RES', 'res_track.txt')

Loading TIFFs: 100%|██████████| 92/92 [00:00<00:00, 709.70it/s]


In [11]:
results = run_metrics(
    gt_data=gt_data, 
    pred_data=pred_data, 
    matcher=CTCMatched, 
    metrics=[CTCMetrics, DivisionMetrics], 
    frame_buffer=(0,1,2)
)

Evaluating edges: 100%|██████████| 8535/8535 [00:06<00:00, 1406.70it/s]


In [9]:
results['CTCMetrics'].results

{'AOGM': 631.5,
 'fp_nodes': 0,
 'fn_nodes': 39,
 'ns_nodes': 0,
 'fp_edges': 60,
 'fn_edges': 87,
 'ws_edges': 51,
 'TRA': 0.9936361895740329,
 'DET': 0.9954855886097927}

In [12]:
results['DivisionMetrics'].results

{'Frame Buffer 0': {'Division Recall': 0.8085106382978723,
  'Division Precision': 0.7169811320754716,
  'Division F1': 0.76,
  'Mitotic Branching Correctness': 0.6129032258064516,
  'Total GT Divisions': 94,
  'True Positive Divisions': 76,
  'False Positive Divisions': 30,
  'False Negative Divisions': 18},
 'Frame Buffer 1': {'Division Recall': 0.8085106382978723,
  'Division Precision': 0.7169811320754716,
  'Division F1': 0.76,
  'Mitotic Branching Correctness': 0.6129032258064516,
  'Total GT Divisions': 94,
  'True Positive Divisions': 76,
  'False Positive Divisions': 30,
  'False Negative Divisions': 18},
 'Frame Buffer 2': {'Division Recall': 0.8085106382978723,
  'Division Precision': 0.7169811320754716,
  'Division F1': 0.76,
  'Mitotic Branching Correctness': 0.6129032258064516,
  'Total GT Divisions': 94,
  'True Positive Divisions': 76,
  'False Positive Divisions': 30,
  'False Negative Divisions': 18}}